In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from nltk.corpus import stopwords
import nltk

In [3]:
# Загрузка текстов из папки
def load_texts_from_folder(folder_path):
    texts = []
    for file_path in Path(folder_path).glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            texts.append(f.read())
    return texts


# Основной процесс
folder_path = Path("e:/cleaned_text")
texts = load_texts_from_folder(folder_path)

In [4]:
# # Путь к текстовому файлу
# text_file = Path("e:/cleaned_json_letter/Лесков Николай. Том 11.txt")

# # Функция для разбиения текста на предложения
# def split_into_sentences(text):
#     # Регулярное выражение для нахождения предложений
#     sentence_endings = r"[.!?]\\s"
#     sentences = re.split(sentence_endings, text)
#     # Убираем пустые строки и лишние пробелы
#     return [sentence.strip() for sentence in sentences if sentence.strip()]

# # Читаем текстовый файл
# with open(text_file, "r", encoding="utf-8") as file:
#     text = file.read()

In [7]:
# Убедимся, что стоп-слова загружены
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

# Разбиение текста на предложения
def split_into_sentences(text):
    sentences = re.split(r'[.!?]', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Разбиваем текст на предложения
sentences_list = split_into_sentences(texts)

# Преобразуем список в массив NumPy
text_data = np.array(sentences_list)

# Пример использования CountVectorizer
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)

# Используем TfidfVectorizer с фильтрацией стоп-слов и биграммами
tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words=russian_stopwords)
feature_matrix = tfidf.fit_transform(text_data)

# Получаем слова и биграммы с их значениями TF-IDF
feature_names = tfidf.get_feature_names_out()
tfidf_scores = feature_matrix.sum(axis=0).A1

# Создаем список из слов/биграмм и их значений
features_with_scores = list(zip(feature_names, tfidf_scores))

# Сортируем по убыванию значимости
sorted_features = sorted(features_with_scores, key=lambda x: x[1], reverse=True)

# Фильтруем элементы: убираем цифры и комбинации букв и цифр
filtered_features = [(feature, score) for feature, score in sorted_features if not re.search(r"\\d", feature)]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexewd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: expected string or bytes-like object, got 'list'

In [ ]:
# Выводим топ-10 самых значимых слов и биграмм
print("Топ-10 значимых слов и биграмм по TF-IDF:")
for feature, score in filtered_features[:10]:
    print(f"{feature}: {score:.4f}")

In [ ]:
# Выводим примерный результат
print("Пример предложений:", text_data[:5])
print("Форма bag_of_words:", bag_of_words.shape)

In [21]:
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from nltk.corpus import stopwords
import nltk

# Убедимся, что стоп-слова загружены
# nltk.download("stopwords")
# russian_stopwords = stopwords.words("russian")

In [22]:
# Загрузка стоп-слов
nltk.download("stopwords")
russian_stopwords = set(stopwords.words("russian"))
custom_stopwords = {"сноска","и", "в", "во","и", "в", "во", "не", "что", "он", "на", "я", \
"с", "со", "как", "а", "то", "все", "она", "так", "его", "но", "да", "ты", "к", "у", "же", "вы", "за",\
 "бы", "по", "только", "ее", "мне", "было", "вот", "от", "меня", "еще", "нет", "о", "из", "ему", "теперь",\
   "когда", "даже", "ну", "ли", "если", "уже", "или", "ни", "быть", "был", "него", "до", "вас",\
     "нибудь", "опять", "уж", "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они", \
     "тут", "где", "есть", "надо", "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", \
     "без", "будто", "чего", "раз", "тоже", "себе", "под", "лесков", "будет", "ж", "тогда", "кто", \
     "этот", "говорил", "того", "потому", "этого", "какой", "совсем", "ним", "здесь", "этом",\
       "один", "почти", "мой", "тем", "чтобы", "нее", "кажется", "сейчас", "были", "куда",\
         "зачем", "сказать", "всех", "никогда", "можно", "при", "наконец", "два",\
           "об", "хоть", "после", "над", "тот", "через", "эти", "нас", "про",\
             "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем", "свою",\
               "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более",\
                 "всегда", "конечно", "всю","это",\
                   "не", "что", "он", "на", "я", "с",\
                     "со", "как", "н", "то", "все", "она", "так", "г", "очень", "стр", "ты", "к", \
                     "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от",\
                       "меня", "еще", "нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг",\
                         "ли", "если", "уже", "или", "ни", "быть", "был", "него", "до", "вас",\
                           "опять", "уж", "вам", "ведь", "там", "потом", "себя", "ничего", "ей", \
                           "может", "они", "тут", "где", "есть", "надо", "ней", "для", "мы", "тебя",\
                             "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", "тоже",\
                               "себе", "под", "будет", "ж", "тогда", "кто", "этот",\
                                 "того", "потому", "этого", "какой", "совсем", "ним", "здесь", "этом", \
                                 "почти", "мой", "тем", "чтобы", "нее", "кажется", \
                                   "были", "куда", "зачем", "сказать", "всех", "никогда",  \
                                   "можно", "при", "наконец", "два", "об", "другой", "хоть", "после",\
                                     "над", "больше", "тот", "через", "эти", "нас", "про", "них", "какая",\
                                       "много", "разве", "три", "эту", "моя", "впрочем", "свою",\
                                         "этой", "перед", "иногда", "чуть", "том",\
                                           "такой", "им", "более", "конечно"}

russian_stopwords.update(custom_stopwords)

# Убедимся, что стоп-слова преобразованы в список
russian_stopwords = list(russian_stopwords)

# Используем TfidfVectorizer
# tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words=russian_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexewd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:

# Функция для разбиения текста на предложения
def split_into_sentences(text):
    # Регулярное выражение для нахождения предложений
    sentence_endings = r"[.!?]\s"
    sentences = re.split(sentence_endings, text)
    # Убираем пустые строки и лишние пробелы
    return [sentence.strip() for sentence in sentences if sentence.strip()]


def load_texts_from_folder(folder_path):
    texts = []
    for file_path in Path(folder_path).glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            texts.append(f.read())
    return texts


# Основной процесс
folder_path = Path("e:/cleaned_text")
texts = load_texts_from_folder(folder_path)

# Разбиваем тексты на предложения и создаём один общий список предложений
all_sentences = []
for text in texts:
    all_sentences.extend(split_into_sentences(text))

# Преобразуем список предложений в массив NumPy
text_data = np.array(all_sentences)

# Пример использования CountVectorizer
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)

# Используем TfidfVectorizer с фильтрацией стоп-слов и биграммами
# tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words=russian_stopwords)
# feature_matrix = tfidf.fit_transform(text_data)

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),  # Односложные слова и биграммы
    stop_words=russian_stopwords,  # Удаляем стоп-слова
    max_df=0.8,  # Убираем слишком частые слова
    min_df=5,    # Убираем слишком редкие слова
    norm="l2"    # Применяем нормализацию
)

# tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words=russian_stopwords)

feature_matrix = tfidf.fit_transform(text_data)


# Получаем слова и биграммы с их значениями TF-IDF
feature_names = tfidf.get_feature_names_out()
tfidf_scores = feature_matrix.sum(axis=0).A1

# Создаем список из слов/биграмм и их значений
features_with_scores = list(zip(feature_names, tfidf_scores))

# Сортируем по убыванию значимости
sorted_features = sorted(features_with_scores, key=lambda x: x[1], reverse=True)

# Фильтруем элементы: убираем цифры и комбинации букв и цифр
filtered_features = [(feature, score) for feature, score in sorted_features if not re.search(r"\d", feature)]

# Выводим топ-10 самых значимых слов и биграмм
print("Топ-10 значимых слов и биграмм по TF-IDF:")
for feature, score in filtered_features[:40]:
    print(f"{feature}: {score:.2f}")

print("Форма bag_of_words:", bag_of_words.shape)


Топ-10 значимых слов и биграмм по TF-IDF:
говорит: 722.92
такое: 569.58
говорю: 567.09
дело: 480.56
тебе: 468.50
знаю: 460.12
время: 429.13
сказал: 400.75
отвечал: 385.82
человек: 382.96
см: 360.29
отчего: 350.67
нам: 321.75
люди: 321.08
который: 318.42
лескова: 314.22
нем: 304.24
могу: 304.12
всем: 303.61
этим: 303.29
спросил: 293.85
делать: 277.66
своей: 276.52
прим: 274.53
однако: 271.31
франц: 271.01
мог: 270.49
людей: 267.94
которые: 260.98
своего: 259.28
отец: 256.46
думаю: 252.01
лиза: 250.78
ах: 241.34
почему: 238.50
письмо: 237.39
говорят: 237.06
кого: 234.63
разумеется: 234.51
просто: 233.10
Форма bag_of_words: (141668, 164543)


In [25]:
# Используем TfidfVectorizer для биграмм и триграмм
tfidf_ngrams = TfidfVectorizer(ngram_range=(2, 3), stop_words=russian_stopwords)
ngram_feature_matrix = tfidf_ngrams.fit_transform(text_data)

# Получаем биграммы и триграммы с их значениями TF-IDF
ngram_feature_names = tfidf_ngrams.get_feature_names_out()
ngram_tfidf_scores = ngram_feature_matrix.sum(axis=0).A1

# Создаем список из биграмм/триграмм и их значений
ngram_features_with_scores = list(zip(ngram_feature_names, ngram_tfidf_scores))

# Сортируем по убыванию значимости
sorted_ngram_features = sorted(ngram_features_with_scores, key=lambda x: x[1], reverse=True)

# Фильтруем биграммы и триграммы (убираем цифры и другие ненужные элементы)
filtered_ngram_features = [
    (feature, score) for feature, score in sorted_ngram_features if not re.search(r"\d", feature)
]

# Выводим топ-10 биграмм и триграмм
print("\nТоп-10 значимых биграмм и триграмм по TF-IDF:")
for feature, score in filtered_ngram_features[:40]:
    print(f"{feature}: {score:.2f}")



Топ-10 значимых биграмм и триграмм по TF-IDF:
публикуется впервые: 229.40
печатается автографу: 124.55
самом деле: 85.64
жизнь николая: 75.94
жизнь николая лескова: 75.94
николая лескова: 75.94
автографу ирли: 72.81
печатается автографу ирли: 72.34
впервые опубликовано: 64.39
собрание сочинений: 60.21
примечание письму: 48.22
лизавета егоровна: 45.69
крайней мере: 40.01
сих пор: 38.29
шестидесятые годы: 36.26
имеется виду: 35.76
катерина львовна: 35.44
домна платоновна: 35.00
тех пор: 32.59
речь идет: 31.20
впервые опубликовано сборнике: 31.00
опубликовано сборнике: 31.00
друг друга: 29.97
черт знает: 29.84
марфа андревна: 29.76
впервые опубликовано книге: 28.82
опубликовано книге: 28.82
автографу цгали: 28.48
бог знает: 28.34
печатается автографу цгали: 28.18
анна семеновна: 28.01
печатается тексту: 27.99
евгения петровна: 26.95
таким образом: 25.41
алексей сергеевич: 24.66
спросил розанов: 24.54
слава богу: 24.19
фирс григорьич: 24.16
ида ивановна: 24.05
сборнике шестидесятые: 23.88

In [30]:
import pandas as pd

# Используем TfidfVectorizer для монограмм
tfidf_monograms = TfidfVectorizer(ngram_range=(1, 1), stop_words=russian_stopwords)
monogram_feature_matrix = tfidf_monograms.fit_transform(text_data)

# Получаем монограммы и их значения TF-IDF
monogram_feature_names = tfidf_monograms.get_feature_names_out()
monogram_tfidf_scores = monogram_feature_matrix.sum(axis=0).A1

# Создаем список из монограмм и их значений
monogram_features_with_scores = list(zip(monogram_feature_names, monogram_tfidf_scores))

# Создаем DataFrame для монограмм
monogram_df = pd.DataFrame(monogram_features_with_scores, columns=["Monogram", "TF-IDF"])
monogram_df["Frequency"] = monogram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота на основе TF-IDF

# Округляем значения до 2 знаков после запятой
monogram_df["TF-IDF"] = monogram_df["TF-IDF"].round(2)
monogram_df["Frequency"] = monogram_df["Frequency"].round(2)

monogram_df = monogram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Используем TfidfVectorizer для биграмм и триграмм
tfidf_ngrams = TfidfVectorizer(ngram_range=(2, 3), stop_words=russian_stopwords)
ngram_feature_matrix = tfidf_ngrams.fit_transform(text_data)

# Получаем биграммы и триграммы с их значениями TF-IDF
ngram_feature_names = tfidf_ngrams.get_feature_names_out()
ngram_tfidf_scores = ngram_feature_matrix.sum(axis=0).A1

# Создаем список из биграмм и триграмм и их значений
ngram_features_with_scores = list(zip(ngram_feature_names, ngram_tfidf_scores))

# Фильтруем биграммы и триграммы (убираем цифры)
filtered_ngram_features = [
    (feature, score) for feature, score in ngram_features_with_scores if not re.search(r"\d", feature)
]

# Разделяем на биграммы и триграммы
bigram_features = [(feature, score) for feature, score in filtered_ngram_features if len(feature.split()) == 2]
trigram_features = [(feature, score) for feature, score in filtered_ngram_features if len(feature.split()) == 3]

# Создаем DataFrame для биграмм
bigram_df = pd.DataFrame(bigram_features, columns=["Bigram", "TF-IDF"])
bigram_df["Frequency"] = bigram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота

# Округляем значения до 2 знаков после запятой
bigram_df["TF-IDF"] = bigram_df["TF-IDF"].round(2)
bigram_df["Frequency"] = bigram_df["Frequency"].round(2)

bigram_df = bigram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Создаем DataFrame для триграмм
trigram_df = pd.DataFrame(trigram_features, columns=["Trigram", "TF-IDF"])
trigram_df["Frequency"] = trigram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота

# Округляем значения до 2 знаков после запятой
trigram_df["TF-IDF"] = trigram_df["TF-IDF"].round(2)
trigram_df["Frequency"] = trigram_df["Frequency"].round(2)

trigram_df = trigram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)


In [31]:
# Выводим таблицы
print("Монограммы:")
print(monogram_df.head(20))  # Топ-10 монограмм

Монограммы:
   Monogram  TF-IDF  Frequency
0   говорит  665.54   66554.06
1     такое  554.60   55459.94
2    говорю  540.90   54089.75
3      дело  457.10   45710.44
4      знаю  452.30   45230.08
5      тебе  449.54   44953.81
6     время  392.09   39208.91
7   отвечал  377.47   37746.52
8    сказал  374.22   37421.68
9   человек  357.34   35733.60
10       см  335.70   33569.62
11   отчего  334.58   33458.33
12  лескова  329.00   32900.01
13     могу  300.05   30004.79
14      нам  290.78   29077.67
15     люди  290.64   29063.97
16  спросил  290.08   29008.28
17  который  283.80   28380.10
18   делать  273.53   27353.03
19     всем  273.20   27319.52


In [32]:
print("\nБиграммы:")
print(bigram_df.head(20))  # Топ-10 биграмм


Биграммы:
                   Bigram  TF-IDF  Frequency
0     публикуется впервые  229.40   22939.59
1    печатается автографу  124.55   12455.21
2              самом деле   85.64    8563.52
3           жизнь николая   75.94    7593.77
4         николая лескова   75.94    7593.77
5          автографу ирли   72.81    7281.25
6    впервые опубликовано   64.39    6438.63
7      собрание сочинений   60.21    6021.29
8       примечание письму   48.22    4821.99
9       лизавета егоровна   45.69    4569.05
10           крайней мере   40.01    4000.58
11                сих пор   38.29    3829.01
12      шестидесятые годы   36.26    3625.90
13           имеется виду   35.76    3575.99
14       катерина львовна   35.44    3544.08
15       домна платоновна   35.00    3499.63
16                тех пор   32.59    3259.08
17              речь идет   31.20    3119.94
18  опубликовано сборнике   31.00    3100.26
19             друг друга   29.97    2996.64


In [33]:
print("\nТриграммы:")
print(trigram_df.head(20))  # Топ-10 триграмм


Триграммы:
                               Trigram  TF-IDF  Frequency
0                жизнь николая лескова   75.94    7593.77
1            печатается автографу ирли   72.34    7234.28
2        впервые опубликовано сборнике   31.00    3100.26
3           впервые опубликовано книге   28.82    2882.17
4           печатается автографу цгали   28.18    2817.53
5           сборнике шестидесятые годы   23.88    2387.62
6             печатается автографу гпб   22.97    2296.86
7   опубликовано сборнике шестидесятые   22.85    2285.45
8            полное собрание сочинений   19.42    1941.96
9                   ваш покорный слуга   13.55    1354.84
10         уважаемый алексей сергеевич    9.81     981.26
11             уважаемый петр карлович    9.73     972.76
12    печатается автографу толстовский    9.69     968.73
13         автографу толстовский музей    9.69     968.73
14            сборнике письма толстого    9.39     938.62
15        опубликовано сборнике письма    9.39     938.62
16

In [45]:
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from pathlib import Path
from nltk.corpus import stopwords
import nltk

# Загрузка текстов из папки
def load_texts_from_folder(folder_path):
    texts = []
    for file_path in Path(folder_path).glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            texts.append(f.read())
    return texts

# Убедимся, что стоп-слова загружены
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

# Разбиение текста на предложения
def split_into_sentences(text):
    sentences = re.split(r'[.!?]', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Основной процесс
folder_path = Path("e:/cleaned_text")
texts = load_texts_from_folder(folder_path)

# Разбиваем каждый текст на предложения
sentences_list = [split_into_sentences(text) for text in texts]

# Преобразуем список в одномерный список предложений
all_sentences = [sentence for sublist in sentences_list for sentence in sublist]

# Преобразуем в NumPy, если нужно
text_data = np.array(all_sentences, dtype=object)

# Функция для очистки биграмм и триграмм
def clean_bigrams_trigrams(text):
    # Убираем символы "_" в начале и конце, а также одиночные буквы
    text = re.sub(r'^\_+|\_+$', '', text)  # Убираем символы "_" в начале и конце
    text = re.sub(r'\b\w{1}\b', '', text)  # Убираем одиночные буквы
    text = re.sub(r'\d+', '', text)  # Убираем цифры
    return text.strip()

# Функция для генерации биграмм и триграмм
def generate_bigrams_trigrams(text_data):
    # Генерация биграмм и триграмм
    vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words=russian_stopwords)
    count_matrix = vectorizer.fit_transform(text_data)
    
    # Извлекаем биграммы и триграммы
    ngram_features = vectorizer.get_feature_names_out()
    
    # Применяем очистку
    cleaned_ngrams = [clean_bigrams_trigrams(ngram) for ngram in ngram_features]
    
    return cleaned_ngrams, count_matrix

# Получаем очищенные биграммы и триграммы
cleaned_bigrams_trigrams, count_matrix = generate_bigrams_trigrams(text_data)

# Печать очищенных биграмм и триграмм
# Печать первых 10 очищенных биграмм и триграмм
print(cleaned_bigrams_trigrams[:40])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexewd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['', 'иностранных', 'иностранных слов', 'руб', 'рублей', 'рублей дохода', 'экземпляров', 'экземпляров концы', '', '', '', '', '', '', '', 'будут', 'затем', 'листов', 'подписью', '', 'гродно', '', 'лист', 'руб', '', 'годах', 'мин', '', 'мая', 'августа', 'августа', 'августа', 'августа буду', 'августа выехал', 'апреля', 'апреля', 'библейского', 'библейского поэта', 'благонамеренная', 'благонамеренная бестактность']


In [50]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re

# Функция для очистки биграмм и триграмм
def clean_bigrams_trigrams(text):
    # Убираем символы "_" в начале и конце, а также одиночные буквы
    text = re.sub(r'^\_+|\_+$', '', text)  # Убираем символы "_" в начале и конце
    text = re.sub(r'\b\w{1}\b', '', text)  # Убираем одиночные буквы
    text = re.sub(r'\d+', '', text)  # Убираем цифры
    text = re.sub(r'[a-zA-Z]', '', text)  # Убираем латинские буквы (верхний и нижний регистр)
    return text.strip()

# Функция для генерации биграмм и триграмм
def generate_bigrams_trigrams(text_data):
    # Генерация биграмм и триграмм
    vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words='english')
    count_matrix = vectorizer.fit_transform(text_data)
    
    # Извлекаем биграммы и триграммы
    ngram_features = vectorizer.get_feature_names_out()
    
    # Применяем очистку
    cleaned_ngrams = [clean_bigrams_trigrams(ngram) for ngram in ngram_features]
    
    return cleaned_ngrams, count_matrix

# Получаем очищенные биграммы и триграммы
cleaned_bigrams_trigrams, count_matrix = generate_bigrams_trigrams(text_data)

# Преобразуем разреженную матрицу в DataFrame для частот
ngram_frequencies = np.array(count_matrix.sum(axis=0)).flatten()

# Разделяем на биграммы и триграммы
bigrams = [ngram for ngram in cleaned_bigrams_trigrams if len(ngram.split()) == 2]
bigrams_frequencies = ngram_frequencies[[i for i, ngram in enumerate(cleaned_bigrams_trigrams) if len(ngram.split()) == 2]]

trigrams = [ngram for ngram in cleaned_bigrams_trigrams if len(ngram.split()) == 3]
trigrams_frequencies = ngram_frequencies[[i for i, ngram in enumerate(cleaned_bigrams_trigrams) if len(ngram.split()) == 3]]

# Создаем pandas таблицы для биграмм и триграмм
bigrams_df = pd.DataFrame({'Bigram': bigrams, 'Frequency': bigrams_frequencies})
trigrams_df = pd.DataFrame({'Trigram': trigrams, 'Frequency': trigrams_frequencies})

# Выводим первые 40 биграмм и триграмм
# print("Bigrams:")
# print(bigrams_df.head(40))

# print("\nTrigrams:")
# print(trigrams_df.head(40))


In [51]:
# Выводим первые 40 биграмм и триграмм
print("Bigrams:")
print(bigrams_df.head(40))

Bigrams:
                          Bigram  Frequency
0               иностранных слов          2
1                  рублей дохода          1
2                 экземпляров во          1
3                   августа буду          1
4                 августа выехал          1
5                    без подписи          1
6              библейского поэта          1
7   благонамеренная бестактность          1
8                       боже мой          2
9                        вот это          2
10                   всем шильям          1
11          вызвал сочувственную          1
12              выражением франц          2
13                 галманы олухи          1
14               гимназия велась          1
15                    го августа          2
16                     го апреля          1
17                    го генваря          1
18                        го мая          1
19                    декабря на          1
20                  желаем знать          1
21                   жи

In [52]:
print("\nTrigrams:")
print(trigrams_df.head(40))


Trigrams:
                            Trigram  Frequency
0   з способствовал археологическим          1
1                 год был министром          1
2                   де бараль сыном          1
3                     т говорит как          1
4         торично флологчного вддлу          1
5                     з розшукв про          1
6    сторично філологічного відділу          1
7                   е бертольди для          1
8               аа кормили отвечала          1
9              ддв великих прауноки          1
10             видом нашим мартыном          1
11                   і чоловіки мав          1
12                     і языки умів          1
13                аиерей может быть          1
14               аарон митре жезлом          1
15                    аарон один из          1
16           аарона воткнутый землю          1
17              аарона дафан авирон          1
18                   аарония во всю          1
19             ааронов отцу захарии          1
20

In [53]:
print(trigrams_df.sample(40))

                               Trigram  Frequency
941064        прочтения известного нам          1
822240             писания но говорить          1
1035255              сию самую несытую          1
360210                    захочу но не          1
776170         от древних владетельных          1
46032                  було десь давно          1
551527            моей доверенности по          1
837035                    по форме под          1
535316                   мира где люди          1
808281           пачками руках кабинет          1
1216687          учился отлично учился          1
703204            новом этапе развития          1
1009086               своим делом было          1
846786                   поди тебе эту          1
956662               разговоре это для          1
838740              повелеть чтобы при          1
4672       альбоме впечатлений заметок          1
747033        он повернулся компрессом          1
712490            об этико философских          1


InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got 'russian' instead.

In [ ]:
import pandas as pd

# Создаем DataFrame для биграмм и триграмм
bigram_df = pd.DataFrame({
    'Bigram/Trigram': cleaned_bigrams_trigrams,
    'Frequency': count_matrix.sum(axis=0).A1
})

# Сохраняем в CSV файл
bigram_df.to_csv("bigrams_trigrams.csv", index=False)

In [69]:
# Получаем частоту для каждого термина (если уже используете CountVectorizer)
count = CountVectorizer(ngram_range=(1, 3), stop_words=russian_stopwords)
count_matrix = count.fit_transform(text_data)

# Преобразуем в DataFrame для удобства
count_df = pd.DataFrame(count_matrix.toarray(), columns=count.get_feature_names_out())

# Получаем частоты для каждого термина (нормализуем, если нужно)
frequency = count_df.sum(axis=0)

# Создаём DataFrame с биграммами и частотой
bigram_df = pd.DataFrame({
    'Bigram': frequency.index,
    'Frequency': frequency.values
})

# Для нормализации, например, делим на количество слов в документе:
# total_words = sum(frequency)
# bigram_df['Normalized Frequency'] = bigram_df['Frequency'] / total_words


MemoryError: Unable to allocate 2.14 TiB for an array with shape (149142, 1973550) and data type int64

In [72]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from pathlib import Path

# Функция для загрузки текстов из папки
def load_texts_from_folder(folder_path):
    texts = []
    for file_path in Path(folder_path).glob("*.txt"):  # Чтение всех текстов с расширением .txt
        with open(file_path, "r", encoding="utf-8") as file:
            texts.append(file.read())
    return texts

# Указываем путь к папке с текстами Н. С. Лескова
folder_path = "e:/cleaned_text"  # Путь к папке, где находятся файлы

# Загружаем все тексты
texts = load_texts_from_folder(folder_path)

# Преобразуем в CountVectorizer (извлечение биграмм)
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words=None)  # Для биграмм
X = vectorizer.fit_transform(texts)

# Получаем все биграммы
ngram_features = vectorizer.get_feature_names_out()

# Подсчитываем частоту биграмм
frequency = X.sum(axis=0).A1  # Преобразуем в одномерный массив

# Преобразуем в pandas Series
frequency_series = pd.Series(frequency, index=ngram_features)

# Сортируем по частоте и ограничиваем количество
top_n = 10  # Ограничиваем до 10 самых частых биграмм
top_bigrams = frequency_series.sort_values(ascending=False).head(top_n)

# Создаем DataFrame с ограниченным количеством биграмм
bigram_df = pd.DataFrame({
    'Bigram': top_bigrams.index,
    'Frequency': top_bigrams.values
})

# Печать результата
print(bigram_df)


       Bigram  Frequency
0  потому что       2210
1      что он       1749
2     что это       1544
3     все это       1386
4  может быть       1268
5   ничего не       1240
6      что же       1210
7      то что       1131
8     не было       1113
9     то есть        963


In [68]:
import re
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

# Загрузка текстов из папки
def load_texts_from_folder(folder_path):
    texts = []
    for file_path in Path(folder_path).glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            texts.append(f.read())
    return texts

# Преобразуем тексты в матрицу термов
def vectorize_texts(texts):
    vectorizer = CountVectorizer(stop_words=russian_stopwords)  # Передаем список стоп-слов
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Функция для поиска похожих слов
def find_similar_words(texts, target_word):
    # Векторизация текста
    X, vectorizer = vectorize_texts(texts)

    # Получаем индексы всех слов
    feature_names = np.array(vectorizer.get_feature_names_out())

    # Проверяем, есть ли целевое слово в словаре
    if target_word not in feature_names:
        print(f"Слово '{target_word}' не найдено в тексте.")
        return

    # Получаем вектор для целевого слова
    target_idx = np.where(feature_names == target_word)[0][0]
    target_vector = X[:, target_idx].toarray().flatten()

    # Считаем косинусное сходство между словом и всеми остальными словами
    similarities = cosine_similarity(X.T, target_vector.reshape(1, -1))

    # Получаем индексы наиболее похожих слов
    similar_idx = similarities.flatten().argsort()[::-1]

    # Печатаем 10 наиболее похожих слов
    print(f"10 наиболее похожих слов к слову '{target_word}':")
    for idx in similar_idx[:40]:
        print(feature_names[idx])

# Загрузка всех текстов
folder_path = Path("e:/cleaned_text")  # Указываем путь к папке с текстами
texts = load_texts_from_folder(folder_path)

# Пример использования
target_word = "любовь"  # Слово для поиска похожих слов
find_similar_words(texts, target_word)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexewd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


10 наиболее похожих слов к слову 'любовь':
любовь
молодых
любви
молодой
жена
старики
устали
платить
движением
никак
временем
ехать
смотреть
ногу
живет
длинное
красивый
раза
пальцы
денег
рукою
нервы
свободно
своими
успокоился
прочие
поле
расположение
свои
никого
зорко
дорогой
дороге
старика
радости
охота
знает
забыл
жене
столом


In [26]:
import pandas as pd

# Используем TfidfVectorizer для монограмм
tfidf_monograms = TfidfVectorizer(ngram_range=(1, 1), stop_words=russian_stopwords)
monogram_feature_matrix = tfidf_monograms.fit_transform(text_data)

# Получаем монограммы и их значения TF-IDF
monogram_feature_names = tfidf_monograms.get_feature_names_out()
monogram_tfidf_scores = monogram_feature_matrix.sum(axis=0).A1

# Создаем список из монограмм и их значений
monogram_features_with_scores = list(zip(monogram_feature_names, monogram_tfidf_scores))

# Создаем DataFrame для монограмм
monogram_df = pd.DataFrame(monogram_features_with_scores, columns=["Monogram", "TF-IDF"])
monogram_df["Frequency"] = monogram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота на основе TF-IDF
monogram_df = monogram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Используем TfidfVectorizer для биграмм и триграмм
tfidf_ngrams = TfidfVectorizer(ngram_range=(2, 3), stop_words=russian_stopwords)
ngram_feature_matrix = tfidf_ngrams.fit_transform(text_data)

# Получаем биграммы и триграммы с их значениями TF-IDF
ngram_feature_names = tfidf_ngrams.get_feature_names_out()
ngram_tfidf_scores = ngram_feature_matrix.sum(axis=0).A1

# Создаем список из биграмм и триграмм и их значений
ngram_features_with_scores = list(zip(ngram_feature_names, ngram_tfidf_scores))

# Фильтруем биграммы и триграммы (убираем цифры)
filtered_ngram_features = [
    (feature, score) for feature, score in ngram_features_with_scores if not re.search(r"\d", feature)
]

# Создаем DataFrame для биграмм и триграмм
ngram_df = pd.DataFrame(filtered_ngram_features, columns=["Ngram", "TF-IDF"])
ngram_df["Frequency"] = ngram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота
ngram_df = ngram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Выводим таблицы
print("Монограммы:")
print(monogram_df.head(20))  # Топ-10 монограмм

print("\nБиграммы и Триграммы:")
print(ngram_df.head(20))  # Топ-10 биграмм и триграмм


Монограммы:
   Monogram      TF-IDF     Frequency
0   говорит  665.540584  66554.058448
1     такое  554.599412  55459.941211
2    говорю  540.897516  54089.751551
3      дело  457.104448  45710.444800
4      знаю  452.300844  45230.084386
5      тебе  449.538056  44953.805623
6     время  392.089052  39208.905215
7   отвечал  377.465160  37746.515957
8    сказал  374.216788  37421.678850
9   человек  357.336025  35733.602496
10       см  335.696229  33569.622867
11   отчего  334.583309  33458.330913
12  лескова  329.000122  32900.012187
13     могу  300.047851  30004.785134
14      нам  290.776728  29077.672773
15     люди  290.639733  29063.973281
16  спросил  290.082771  29008.277101
17  который  283.800969  28380.096932
18   делать  273.530325  27353.032521
19     всем  273.195163  27319.516322

Биграммы и Триграммы:
                        Ngram      TF-IDF     Frequency
0         публикуется впервые  229.395895  22939.589518
1        печатается автографу  124.552079  12455.207882

In [27]:
import pandas as pd

# Используем TfidfVectorizer для монограмм
tfidf_monograms = TfidfVectorizer(ngram_range=(1, 1), stop_words=russian_stopwords)
monogram_feature_matrix = tfidf_monograms.fit_transform(text_data)

# Получаем монограммы и их значения TF-IDF
monogram_feature_names = tfidf_monograms.get_feature_names_out()
monogram_tfidf_scores = monogram_feature_matrix.sum(axis=0).A1

# Создаем список из монограмм и их значений
monogram_features_with_scores = list(zip(monogram_feature_names, monogram_tfidf_scores))

# Создаем DataFrame для монограмм
monogram_df = pd.DataFrame(monogram_features_with_scores, columns=["Monogram", "TF-IDF"])
monogram_df["Frequency"] = monogram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота на основе TF-IDF
monogram_df = monogram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Используем TfidfVectorizer для биграмм и триграмм
tfidf_ngrams = TfidfVectorizer(ngram_range=(2, 3), stop_words=russian_stopwords)
ngram_feature_matrix = tfidf_ngrams.fit_transform(text_data)

# Получаем биграммы и триграммы с их значениями TF-IDF
ngram_feature_names = tfidf_ngrams.get_feature_names_out()
ngram_tfidf_scores = ngram_feature_matrix.sum(axis=0).A1

# Создаем список из биграмм и триграмм и их значений
ngram_features_with_scores = list(zip(ngram_feature_names, ngram_tfidf_scores))

# Фильтруем биграммы и триграммы (убираем цифры)
filtered_ngram_features = [
    (feature, score) for feature, score in ngram_features_with_scores if not re.search(r"\d", feature)
]

# Разделяем на биграммы и триграммы
bigram_features = [(feature, score) for feature, score in filtered_ngram_features if len(feature.split()) == 2]
trigram_features = [(feature, score) for feature, score in filtered_ngram_features if len(feature.split()) == 3]

# Создаем DataFrame для биграмм
bigram_df = pd.DataFrame(bigram_features, columns=["Bigram", "TF-IDF"])
bigram_df["Frequency"] = bigram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота
bigram_df = bigram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Создаем DataFrame для триграмм
trigram_df = pd.DataFrame(trigram_features, columns=["Trigram", "TF-IDF"])
trigram_df["Frequency"] = trigram_df["TF-IDF"].apply(lambda x: x * 100)  # Примерная частота
trigram_df = trigram_df.sort_values(by="TF-IDF", ascending=False).reset_index(drop=True)

# Выводим таблицы
print("Монограммы:")
print(monogram_df.head(20))  # Топ-10 монограмм


Монограммы:
   Monogram      TF-IDF     Frequency
0   говорит  665.540584  66554.058448
1     такое  554.599412  55459.941211
2    говорю  540.897516  54089.751551
3      дело  457.104448  45710.444800
4      знаю  452.300844  45230.084386
5      тебе  449.538056  44953.805623
6     время  392.089052  39208.905215
7   отвечал  377.465160  37746.515957
8    сказал  374.216788  37421.678850
9   человек  357.336025  35733.602496
10       см  335.696229  33569.622867
11   отчего  334.583309  33458.330913
12  лескова  329.000122  32900.012187
13     могу  300.047851  30004.785134
14      нам  290.776728  29077.672773
15     люди  290.639733  29063.973281
16  спросил  290.082771  29008.277101
17  который  283.800969  28380.096932
18   делать  273.530325  27353.032521
19     всем  273.195163  27319.516322


In [28]:
print("\nБиграммы:")
print(bigram_df.head(20))  # Топ-10 биграмм


Биграммы:
                   Bigram      TF-IDF     Frequency
0     публикуется впервые  229.395895  22939.589518
1    печатается автографу  124.552079  12455.207882
2              самом деле   85.635199   8563.519884
3         николая лескова   75.937683   7593.768255
4           жизнь николая   75.937683   7593.768255
5          автографу ирли   72.812541   7281.254137
6    впервые опубликовано   64.386294   6438.629368
7      собрание сочинений   60.212877   6021.287704
8       примечание письму   48.219905   4821.990488
9       лизавета егоровна   45.690499   4569.049903
10           крайней мере   40.005751   4000.575147
11                сих пор   38.290056   3829.005619
12      шестидесятые годы   36.259000   3625.900009
13           имеется виду   35.759862   3575.986180
14       катерина львовна   35.440780   3544.078020
15       домна платоновна   34.996347   3499.634670
16                тех пор   32.590781   3259.078091
17              речь идет   31.199367   3119.936728
1

In [29]:
print("\nТриграммы:")
print(trigram_df.head(20))  # Топ-10 триграмм


Триграммы:
                               Trigram     TF-IDF    Frequency
0                жизнь николая лескова  75.937683  7593.768255
1            печатается автографу ирли  72.342836  7234.283550
2        впервые опубликовано сборнике  31.002628  3100.262757
3           впервые опубликовано книге  28.821684  2882.168376
4           печатается автографу цгали  28.175346  2817.534641
5           сборнике шестидесятые годы  23.876243  2387.624265
6             печатается автографу гпб  22.968583  2296.858301
7   опубликовано сборнике шестидесятые  22.854501  2285.450140
8            полное собрание сочинений  19.419614  1941.961394
9                   ваш покорный слуга  13.548370  1354.836955
10         уважаемый алексей сергеевич   9.812641   981.264117
11             уважаемый петр карлович   9.727565   972.756467
12    печатается автографу толстовский   9.687288   968.728840
13         автографу толстовский музей   9.687288   968.728840
14            сборнике письма толстого   9.

In [12]:

# Выводим топ-10 самых значимых слов и биграмм
print("Топ-10 значимых слов и биграмм по TF-IDF:")
for feature, score in filtered_features[:40]:
    print(f"{feature}: {score:.4f}")


Топ-10 значимых слов и биграмм по TF-IDF:
это: 0.6229
очень: 0.2434
лесков: 0.1868
стр: 0.1290
лескова: 0.1209
печатается: 0.1061
письмо: 0.1039
петербург: 0.1018
впервые: 0.0946
время: 0.0894
автографу: 0.0857
печатается автографу: 0.0841
который: 0.0801
года: 0.0739
толстого: 0.0705
кажется: 0.0690
дело: 0.0680
письма: 0.0674
нем: 0.0628
см: 0.0628
которые: 0.0619
могу: 0.0612
знаю: 0.0551
публикуется: 0.0532
публикуется впервые: 0.0532
человек: 0.0523
рассказ: 0.0510
люди: 0.0507
вами: 0.0492
ваше: 0.0492
которого: 0.0467
этим: 0.0467
людей: 0.0464
которых: 0.0448
статьи: 0.0448
ваш: 0.0436
времени: 0.0436
жизни: 0.0436
ирли: 0.0433
публикация: 0.0433


In [11]:

# Выводим примерный результат
# print("Пример предложений:", text_data[:5])
print("Форма bag_of_words:", bag_of_words.shape)

Форма bag_of_words: (1, 38043)
